# Imports

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

from google.colab import files
from google.colab import drive
drive.mount("/content/gdrive")

import pandas as pd
import io
import numpy as np


# Dataset import

In [5]:
directory = '/content/gdrive/My Drive/Colab Notebooks/'

# train dataset impot
df = pd.read_csv(directory+'NLP_Project - Sheet1.csv')
df = df[df.columns.difference(['Comments'])]
df = df[['Text', 'Summary']]
pd.set_option("max_columns", None) # show all cols
pd.set_option('max_colwidth', None) # show full width of showing cols
pd.set_option("expand_frame_repr", False) # print cols side by side as it's supposed to be
df.loc[[0]]

,Text,Summary
0,"After checking out of our Airbnb, we drove just one hour up the road to Wanaka, where we grabbed some coffee and breakfast and checked a few items off our bucket list (including #ThatWanakaTree).\n\nFrom there, it was a 2.5-hour journey by car to Mount Cook Village—one of the destinations I had been most looking forward to. We stopped just twice more along the way: once at Lindis Pass Lookout, and again at Peter’s Lookout near Lake Pukaki.\n\nLindis Pass actually reminded me a bit of California! The landscape was incredibly brown, and the hills were quite low and rolling. It’s a good place to stretch your legs and take a break from driving, but our stop here was short, as there’s not too much to see.\n\nFrom here, it’s only around 35 minutes more to Mount Cook Village—and the views get a whole lot more epic with each passing mile.\n\nWe completed the Hooker Valley Track early in the morning during our second day in the park; we were on the trail by 7:00am, and wrapped up our hike before 10:30am.\n\nLater that afternoon, we also completed the Tasman Glacier View Walk.",Drive to Wanaka\nStop by #ThatWanakaTree\nVisit Mount Cook Village\nStop by Lindis Pass Lookout\nStop by Peter's Lookout\nHike Hooker valley Track\nComplete Tasman Glacier View Walk


In [6]:
df

,Text,Summary
0,"After checking out of our Airbnb, we drove just one hour up the road to Wanaka, where we grabbed some coffee and breakfast and checked a few items off our bucket list (including #ThatWanakaTree).\n\nFrom there, it was a 2.5-hour journey by car to Mount Cook Village—one of the destinations I had been most looking forward to. We stopped just twice more along the way: once at Lindis Pass Lookout, and again at Peter’s Lookout near Lake Pukaki.\n\nLindis Pass actually reminded me a bit of California! The landscape was incredibly brown, and the hills were quite low and rolling. It’s a good place to stretch your legs and take a break from driving, but our stop here was short, as there’s not too much to see.\n\nFrom here, it’s only around 35 minutes more to Mount Cook Village—and the views get a whole lot more epic with each passing mile.\n\nWe completed the Hooker Valley Track early in the morning during our second day in the park; we were on the trail by 7:00am, and wrapped up our hike before 10:30am.\n\nLater that afternoon, we also completed the Tasman Glacier View Walk.",Drive to Wanaka\nStop by #ThatWanakaTree\nVisit Mount Cook Village\nStop by Lindis Pass Lookout\nStop by Peter's Lookout\nHike Hooker valley Track\nComplete Tasman Glacier View Walk
1,"Your 3 day Yosemite itinerary begins with a long drive into the park. Starting from either San Francisco, San Jose or Oakland, you’re looking at a 3.5 – 4.5 hour drive, depending on your exact starting point and traffic conditions.\n\nIt’s in your best interest to arrive to the park as early as possible, but as long as you arrive by late afternoon you’ll still be able to follow this itinerary. Stop by Tunnel View on your way into the park; it will be directly on your route if you’re taking the south entrance into the park, or is just a short detour away if you’re taking the north entrance.\nAfter snapping a few photos and soaking in the sweeping scenery at Tunnel View, we continued on to Yosemite Valley Lodge and immediately checked into our room. We were eager to get the car parked and unpacked so we could start enjoying our long weekend in Yosemite!\n\nFrom the lodge, it was a short walk down the road to one of the valley’s easiest and most picturesque “hikes” – Cook’s Meadow. This flat, scenic 1-mile trail yields amazing views of Yosemite Falls and Half Dome right from the valley floor, and is the perfect way to get acquainted with the park.\n\nOn our way back to the lodge, we also did a quick stroll along the Lower Yosemite Falls trail.\n\nToday is the big day! You’ve got a full day to spend in Yosemite National Park, so let’s make the most of it. One of the very best ways to truly explore Yosemite is on your own two feet. We opted for the Upper Yosemite Falls Trail, mostly because the trailhead was a short walk down the road from our lodge. We were on the trail by 6:30am. By 1pm, we were back in the Yosemite Valley Lodge parking lot.\n\nAfter our strenuous hike the day before, it’s safe to say Derek and I weren’t up for any more physical activity. So instead, we opted for a scenic drive up to Glacier Point, home to some of the most dramatic views in all of Yosemite National Park.\n\nFrom way up here above the valley, you can gaze upon iconic sights like Half Dome and Yosemite Falls, and see all the way across to Yosemite’s high country. From Glacier Point, the drive home to San Jose took us around 4.5 hours through the park’s south entrance.\n",Enter into Yosemite National Park\nStop by Tunnel View.\nShort walk down the road to Cook's Meadow\nStroll along the lower Yoselimte Falls Trail\nHike Upper Yosemite Falls Trail\nDrive up to Glacier Point\nGaze upon Half Dome
2,"Tahiti is among the most famous and iconic of French Polynesia’s 67 inhabited islands; the name alone conjures images of palm trees dancing in a warm island breeze, and mai tais being sipped in the moonlight. If you’re not already familiar with travel in French Polynesia, it may surprise you to 

# Preprocessing

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import re

In [8]:
## Preprocess

def remove_stopwords(sent):
  #print(sent)
  # initializing punctuations string
  punc = '''!()-[]{};:'"\,<>/?@#$%^&*_~'''
 
  # Removing punctuations in string
  # Using loop + punctuation string
  for ele in sent:
      if ele in punc:
          sent = sent.replace(ele, "")

  stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(sent)
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  
  #filtered_sentence = [w for w in word_tokens if w.isalnum()] 
  filtered_sentence = ' '.join(filtered_sentence)
  return sent
  #return filtered_sentence

def rem_single_char_space_text(sen):
    # Single character removal
    sentence = re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', sen)
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

def remove_white_spaces(sent):
  sent = re.sub('\s+',' ',sent)
  return sent

In [ ]:
def process_postag(sent):
    #sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

In [10]:
def lemmatize_sentence(sent):
    tokens = nltk.word_tokenize(sent)
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    lemmatized_sentence = ' '.join(lemmatized_sentence)
    #print(lemmatized_sentence)
    return lemmatized_sentence


In [11]:
df['Text'] = df['Text'].apply(str)
df['Text'] = df['Text'].apply(remove_white_spaces)
#df['Text'] = df['Text'].apply(remove_stoprwords)
#df['Text'] = df['Text'].apply(rem_single_char_space_text)
df['Text'] = df['Text'].apply(lemmatize_sentence)
#df['POSTAG_txt'] = df['lem_Text'].apply(process_postag)

# Ground Truth

In [22]:
reference_f = {}
count = 0
for each_corpus in df['Summary']:
  lines = each_corpus.splitlines()
  ref = []
  for i in range(len(lines)):
    ref.append(lines[i].split())
  reference_f[count] = ref
  count = count+1
reference_f

{0: [['Drive', 'to', 'Wanaka'],
  ['Stop', 'by', '#ThatWanakaTree'],
  ['Visit', 'Mount', 'Cook', 'Village'],
  ['Stop', 'by', 'Lindis', 'Pass', 'Lookout'],
  ['Stop', 'by', "Peter's", 'Lookout'],
  ['Hike', 'Hooker', 'valley', 'Track'],
  ['Complete', 'Tasman', 'Glacier', 'View', 'Walk']],
 1: [['Enter', 'into', 'Yosemite', 'National', 'Park'],
  ['Stop', 'by', 'Tunnel', 'View.'],
  ['Short', 'walk', 'down', 'the', 'road', 'to', "Cook's", 'Meadow'],
  ['Stroll', 'along', 'the', 'lower', 'Yoselimte', 'Falls', 'Trail'],
  ['Hike', 'Upper', 'Yosemite', 'Falls', 'Trail'],
  ['Drive', 'up', 'to', 'Glacier', 'Point'],
  ['Gaze', 'upon', 'Half', 'Dome']],
 2: [['Arrive', 'in', "Pape'ete"],
  ['Spend', 'time', 'around', 'Downtown', "Pape'ete"],
  ['Stop', 'by', 'Polynesian', 'Notre', 'Dame', 'Cathedral'],
  ['Visit', "Pape'ete", 'Town', 'Hall'],
  ['Visit', "Pape'ete", 'Market'],
  ['Go', 'to', 'Taharuu', 'Beach'],
  ['Visit', 'Water', 'Gardens', 'of', 'Vaipahi'],
  ['Reach', 'the', 'Isthmus'

# Sentence tokenization

In [14]:
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def tokenize_sentence(sent):
  return sent_tokenize(sent)

df['sent_token_text'] = df['Text'].apply(tokenize_sentence)
df['sent_token_text'][0]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['After check out of our Airbnb , we drive just one hour up the road to Wanaka , where we grab some coffee and breakfast and check a few item off our bucket list ( include # ThatWanakaTree ) .',
 'From there , it be a 2.5-hour journey by car to Mount Cook Village—one of the destination I have be most look forward to .',
 'We stop just twice more along the way : once at Lindis Pass Lookout , and again at Peter ’ s Lookout near Lake Pukaki .',
 'Lindis Pass actually remind me a bit of California !',
 'The landscape be incredibly brown , and the hill be quite low and roll .',
 'It ’ s a good place to stretch your leg and take a break from drive , but our stop here be short , as there ’ s not too much to see .',
 'From here , it ’ s only around 35 minute more to Mount Cook Village—and the view get a whole lot more epic with each pass mile .',
 'We complete the Hooker Valley Track early in the morning during our second day in the park ; we be on the trail by 7:00am , and wrap up our hike be

# Baseline Method: Dependency Parsing

In [20]:
# concatenate NNP
# Output corresponding VBP with it associated NNP

from nltk.parse.stanford import StanfordDependencyParser

path_to_jar = directory+'stanford-parser.jar' 
path_to_models_jar =  directory+'stanford-parser-4.2.0-models.jar'

dependency_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)

#result = dependency_parser.raw_parse('After checking out of our Airbnb we drove just one hour up the road to Wanaka')
#result = dependency_parser.raw_parse('Your 3 day Yosemite itinerary begins with a long drive into the park. Starting from either San Francisco, San Jose or Oakland, you’re looking at a 3.5 – 4.5 hour drive, depending on your exact starting point and traffic conditions. It’s in your best interest to arrive to the park as early as possible, but as long as you arrive by late afternoon you’ll still be able to follow this itinerary. Stop by Tunnel View on your way into the park; it will be directly on your route if you’re taking the south entrance into the park, or is just a short detour away if you’re taking the north entrance.')
def show_dep_parser(sent):
  NNP_VB_container = []
  result = dependency_parser.raw_parse(sent)  
  dep = result.__next__()
  #print(list(dep.triples()))
  for each_comp in list(dep.triples()):
    first_tuple = ','.join(each_comp[0])
    second_tuple = ','.join(each_comp[2])
    if first_tuple.find('NNP') != -1 or second_tuple.find('NNP') != -1:
      if first_tuple.find('VB') != -1 or second_tuple.find('VB') != -1:
        placename = ""
        verb = ""
        if first_tuple.find('NNP') != -1:
            placename = first_tuple.split(',')[0]
            verb = second_tuple.split(',')[0]
        else:
            placename = second_tuple.split(',')[0]
            verb = first_tuple.split(',')[0]
        if verb.lower() == 'remind':
          continue
        print(verb, placename)
        itinerary = (verb, placename)
        #itinerary = verb+" "+full_placename
        NNP_VB_container.append(itinerary)
        #NNP_VB_container.append(each_comp)
  return NNP_VB_container


In [21]:
parsed_tree_f = {}
count = 0
for each_row in df['sent_token_text']:
  #print(each_row)
  print("processing row no ", count)
  parsed_tree = []
  for each_corpus in each_row:
    # inner for loop
    #print(each_corpus)
    parsed_tree.append(show_dep_parser(each_corpus))
  parsed_tree_f[count] = parsed_tree
  count = count+1


processing row no  0
drive Wanaka
grab Wanaka
complete Walk
processing row no  1
depend Francisco
Starting Francisco
depend Jose
Stop View
continue Lodge
enjoy Yosemite
spend Park
opt Trail
weren Derek
gaze Dome
take Point
processing row no  2
see Bora
ete ’
visit Polynesia
felt Polynesia
ete ’
spend Plan
fill Pape
be Beach
soak Taharuu
reach Isthmus
connect Isthmus
connect Nui
connect Iti
follow Tautira
have Tautira
include Stevenson
make Iti
reach Pape
ete Pape
pas Blowhole
processing row no  3
spend Moorea
base Bay
see Church
have Bay
anchor Cook
Located Belvedere
know Titiroa
Continuing Belvedere
Visiting Factory
know Beach
know Beach
attend Center
processing row no  4
build Cathedral
know Cross
include Armas
include East
find Degollado
sit Armas
get Sacramento
backtrack Cathedral
processing row no  5
visit Park
visit Wyoming
processing row no  6
visit Miami
visit States
celebrate Marketplace
visit Miami
hit Centre
end KeyWest
miss States
be Lauderdale
processing row no  7
have Eur

In [23]:
parsed_tree_f

{0: [[('drive', 'Wanaka'), ('grab', 'Wanaka')],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [('complete', 'Walk')]],
 1: [[],
  [('depend', 'Francisco'), ('Starting', 'Francisco'), ('depend', 'Jose')],
  [],
  [('Stop', 'View')],
  [('continue', 'Lodge')],
  [('enjoy', 'Yosemite')],
  [],
  [],
  [],
  [],
  [('spend', 'Park')],
  [],
  [('opt', 'Trail')],
  [],
  [],
  [('weren', 'Derek')],
  [],
  [('gaze', 'Dome')],
  [('take', 'Point')]],
 2: [[],
  [('see', 'Bora')],
  [('ete', '’'), ('visit', 'Polynesia'), ('felt', 'Polynesia')],
  [('ete', '’')],
  [('spend', 'Plan')],
  [],
  [('fill', 'Pape')],
  [],
  [],
  [('be', 'Beach')],
  [('soak', 'Taharuu')],
  [],
  [('reach', 'Isthmus'),
   ('connect', 'Isthmus'),
   ('connect', 'Nui'),
   ('connect', 'Iti')],
  [],
  [],
  [('follow', 'Tautira')],
  [('have', 'Tautira'), ('include', 'Stevenson')],
  [('make', 'Iti')],
  [('reach', 'Pape'), ('ete', 'Pape'), ('pas', 'Blowhole')],
  []],
 3: [[('spend', 'Moorea')],
  [('base', 'Bay')]

In [24]:
def process_output(parsed_tree_f):
  output = {}
  for key, value in parsed_tree_f.items():
    arr = []
    for i in range(len(value)):
      for j in range(len(value[i])):
        arr.append(value[i][j])
      #print(value[i])
    output[key] = arr
    #print(output)
  return output

# Baseline method: BLEU Score

In [25]:
def calculate_BLEU_score(output, reference):
  candidate = []
  sum = 0
  count = 0
  for key, value in output.items():
    for i in range(len(value)):
      candidate = []
      candidate.append(value[i][0])
      candidate.append(value[i][1])
      BLEUscore = nltk.translate.bleu_score.sentence_bleu(reference[key], candidate)
      if BLEUscore > 0:
        count = count + 1
        sum = sum+BLEUscore
        #print(key, BLEUscore)
    #break
  avr = sum*1.0/count
  return avr

In [26]:
output = process_output(parsed_tree_f)
BLEU_score = calculate_BLEU_score(output, reference_f)
print(BLEU_score)

0.485290139048028


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


# Proposed Method

In [27]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from spacy.symbols import nsubj, VERB
nlp = en_core_web_sm.load()

named_entity_list = {}
global_count = 0

# Tokenization using Named Entity

In [31]:
def reconstruct_tokens(sent):
  UNR_ENTITY = ['TIME', 'DATE', 'CARDINAL']
  UNR_ENTITY_INDEX = []
  doc = nlp(sent)
  global named_entity_list
  global global_count
  named_entity_str = {}
  for index, X in enumerate(doc.ents):
    if X.label_ in UNR_ENTITY:
      UNR_ENTITY_INDEX.append(index)
    name = ""
    count = 0
    for words in X:
      if count > 0:
        name += " "
      name += str(words)
      count = count + 1
    named_entity_str[index] = name

  named_entity_list[global_count] = named_entity_str
  global_count = global_count+1
  token = {}
  for index, X in enumerate(doc):
    token[index] = str(X)

  i = 0
  k = 0
  while i<len(token) and k < len(named_entity_str):
    if k in UNR_ENTITY_INDEX:
      k = k+1
      continue
    if token[i] in named_entity_str[k]:
      # match found
      count = 0
      #desired_together_words = ""
      w = named_entity_str[k].split()
      flag = 0
      for words in w:
        if token[i] == words:
          flag = 1
          break
      if flag == 1:
        for words in w:
          token[i] = token[i]+"_"+str(k)
          i = i+1
        k = k+1
      else:
        i = i+1
    else:
      i = i+1

  return token

In [32]:
def reconstruct_sent(sent):
  token = reconstruct_tokens(sent)
  mod_sent = " ".join(list(token.values()))
  return mod_sent

In [33]:
df['Text_ent'] = df['Text'].apply(reconstruct_sent)
df['sent_token_text_ent'] = df['Text_ent'].apply(tokenize_sentence)

In [34]:
df['sent_token_text_ent'][1]

['Your 3 day Yosemite itinerary begin with a long drive into the park .',
 'Starting from either San_1 Francisco_1 , San_2 Jose_2 or Oakland_3 , you ’ re look at a 3.5 – 4.5 hour drive , depend on your exact starting point and traffic condition .',
 'It ’ s in your best interest to arrive to the park as early as possible , but as long as you arrive by late afternoon you ’ ll still be able to follow this itinerary .',
 'Stop by Tunnel_7 View_7 on your way into the park ; it will be directly on your route if you ’ re take the south entrance into the park , or be just a short detour away if you ’ re take the north entrance .',
 'After snap a few photo and soak in the sweeping scenery at Tunnel_8 View_8 , we continue on to Yosemite_9 Valley_9 Lodge_9 and immediately check into our room .',
 'We be eager to get the car park and unpack so we could start enjoy our long weekend in Yosemite_11 !',
 'From the lodge , it be a short walk down the road to one of the valley ’_12 s_12 easy_12 and_12 

# Dependency parsing with named entity tokens

In [35]:
def show_dep_parser_ent(sent, ind):
  NNP_VB_container = []
  global named_entity_list
  named_entity_str = named_entity_list[ind]
  result = dependency_parser.raw_parse(sent)  
  dep = result.__next__()
  for each_comp in list(dep.triples()):
    if each_comp is not None:
      first_tuple = ','.join(each_comp[0])
      second_tuple = ','.join(each_comp[2])
      if first_tuple.find('NNP') != -1 or second_tuple.find('NNP') != -1:
        if first_tuple.find('VB') != -1 or second_tuple.find('VB') != -1:
          placename = ""
          full_placename = ""
          verb = ""
          if first_tuple.find('NNP') != -1:
            placename = first_tuple.split(',')[0]
            verb = second_tuple.split(',')[0]
          else:
            placename = second_tuple.split(',')[0]
            verb = first_tuple.split(',')[0]
          if '_' in placename:
            place_ent_ind = placename.split('_')[1]
            if place_ent_ind.isnumeric():
              full_placename = named_entity_str[int(place_ent_ind)]
            else:
              full_placename = placename
          else:
            full_placename = placename
          if verb.lower() == 'remind':
            continue
          print(verb, full_placename)
          itinerary = (verb, full_placename)
          #itinerary = verb+" "+full_placename
          NNP_VB_container.append(itinerary)
  return NNP_VB_container

In [38]:
ent_parsed_tree_f = {}
count = 0
for each_row in df['sent_token_text_ent']:
  print("processing row ", count)
  parsed_tree = []
  for each_corpus in each_row:
    parsed_tree.append(show_dep_parser_ent(each_corpus, count))
  ent_parsed_tree_f[count] = parsed_tree
  count = count+1

processing row  0
complete the Tasman Glacier View Walk
complete Walk
processing row  1
depend San Francisco
Starting San Francisco
depend San Jose
Stop Tunnel View
continue Yosemite Valley Lodge
enjoy Yosemite
s Meadow
spend Park
opt the Upper Yosemite Falls Trail
walk the Yosemite Valley Lodge
gaze Half Dome
take Glacier Point
processing row  2
see Bora Bora
ete ’
visit Polynesia
felt Polynesia
ete ’
explore Plan
spend Plan
fill Pape
be Taharuu Beach
soak Taharuu
reach Isthmus
connect Isthmus
connect Tahiti Nui
connect Tahiti Iti
isn_30 Tahiti Iti
have Tautira
include James Cook
make Tahiti Iti
reach Pape
ete Pape
pas Blowhole
processing row  3
spend Moorea
see the Saint Joseph Catholic Church
have Opunohu Bay
anchor James Cook
know Marae Titiroa
Visiting Factory
know Mareto Beach
know Beach
attend Center
processing row  4
build Guadalajara Cathedral
know Cross
include Plaza de Armas
find Teatro Degollado
sit the Plaza de Armas
get Sacramento
walk Hospico Cabañas
processing row  5
vi

In [40]:
ent_parsed_tree_f

{0: [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [('complete', 'the Tasman Glacier View Walk')],
  [('complete', 'Walk')]],
 1: [[],
  [('depend', 'San Francisco'),
   ('Starting', 'San Francisco'),
   ('depend', 'San Jose')],
  [],
  [('Stop', 'Tunnel View')],
  [('continue', 'Yosemite Valley Lodge')],
  [('enjoy', 'Yosemite')],
  [('s', 'Meadow')],
  [],
  [],
  [],
  [('spend', 'Park')],
  [],
  [('opt', 'the Upper Yosemite Falls Trail'),
   ('walk', 'the Yosemite Valley Lodge')],
  [],
  [],
  [],
  [],
  [('gaze', 'Half Dome')],
  [('take', 'Glacier Point')]],
 2: [[],
  [('see', 'Bora Bora')],
  [('ete', '’'), ('visit', 'Polynesia'), ('felt', 'Polynesia')],
  [('ete', '’')],
  [('explore', 'Plan'), ('spend', 'Plan')],
  [],
  [('fill', 'Pape')],
  [],
  [],
  [('be', 'Taharuu Beach')],
  [('soak', 'Taharuu')],
  [],
  [('reach', 'Isthmus'),
   ('connect', 'Isthmus'),
   ('connect', 'Tahiti Nui'),
   ('connect', 'Tahiti Iti')],
  [('isn_30', 'Tahiti Iti')],
  [],
  [],
  [('have', '

# Proposed Method: BLEU Score



In [41]:
def process_output_ent(parsed_tree_f):
  output = {}
  for key, value in parsed_tree_f.items():
    arr = []
    for i in range(len(value)):
      for j in range(len(value[i])):
        verb_place_tuple = ()
        place = value[i][j][1]
        place_split = place.split(" ")
        verb_place_tuple = verb_place_tuple+(value[i][j][0],)
        for s in place_split:
          verb_place_tuple = verb_place_tuple+(s,)
        arr.append(verb_place_tuple)
    output[key] = arr
  return output

In [42]:
def calculate_BLEU_score_ent(output, reference):
  candidate = []
  sum = 0
  count = 0
  for key, value in output.items():
    for i in range(len(value)):
      candidate = value[i]
      BLEUscore = nltk.translate.bleu_score.sentence_bleu(reference[key], candidate)
      if BLEUscore > 0:
        count = count + 1
        sum = sum+BLEUscore
        #print(key, BLEUscore)
    #break
  
  avr = sum*1.0/count
  return avr

In [43]:
output_ent = process_output_ent(ent_parsed_tree_f)
prop_BLEU_score = calculate_BLEU_score_ent(output_ent, reference_f)
print(prop_BLEU_score)

0.6036385741883921


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


# Event extraction

In [44]:
processed_docs = list(nlp.pipe(df['Text']))
processed_docs[0]

After check out of our Airbnb , we drive just one hour up the road to Wanaka , where we grab some coffee and breakfast and check a few item off our bucket list ( include # ThatWanakaTree ) . From there , it be a 2.5-hour journey by car to Mount Cook Village—one of the destination I have be most look forward to . We stop just twice more along the way : once at Lindis Pass Lookout , and again at Peter ’ s Lookout near Lake Pukaki . Lindis Pass actually remind me a bit of California ! The landscape be incredibly brown , and the hill be quite low and roll . It ’ s a good place to stretch your leg and take a break from drive , but our stop here be short , as there ’ s not too much to see . From here , it ’ s only around 35 minute more to Mount Cook Village—and the view get a whole lot more epic with each pass mile . We complete the Hooker Valley Track early in the morning during our second day in the park ; we be on the trail by 7:00am , and wrap up our hike before 10:30am . Later that afte

In [45]:
verb_list = ["drive", "visit", "stop", "enter", "hike", "walk", "pass", "be", 'reach'] 
dobj_list = ["village", "view", "park", "road", "volcano", 'islands', 'airport', 'church', 'centre', 'beach', 'bay', 'crossroad'] 

def detect_event(doc, verb_list, dobj_list): 
    for word in doc: 
        #print(word, word.dep_, word.lemma_, word.children)
        if word.dep_ == "ROOT" and word.lemma_ in verb_list: 
            for subword in word.children: 
                #print(subword, subword.dep_)
                if subword.dep_ == "dobj" and subword.lemma_ in dobj_list: 
                    #print(subword)
                    return word, subword

In [46]:
for doc in processed_docs: 
  #print(doc)
  root = detect_event(doc, verb_list, dobj_list) 
  #print(root, obj)
  if root: 
    print(detect_event(doc, verb_list, dobj_list))

(pass, church)
(reach, crossroad)
(visit, beach)
(visit, volcano)
